[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Machine Learning - Supervised Learning - Ensemble Methods - Random Forest

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.002 | 18/03/2025 | Royi Avital | Added the feature `passTicket`                                     |
| 1.0.001 | 14/04/2024 | Royi Avital | Added links to data description                                    |
| 1.0.000 | 08/04/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0054EnsembleRandomForest.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

# Miscellaneous
import math
import os
from platform import python_version
import random

# Typing
from typing import Callable, Dict, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Courses Packages


In [ ]:
# General Auxiliary Functions


## Random Forest Classification

In this note book we'll use the _Random Forest_ based classifier in the task of estimating whether a passenger on the Titanic will or will not survive.  
We'll focus on basic pre processing of the data and analyzing the importance of features using the classifier.

* <font color='brown'>(**#**)</font> This is a very popular data set for classification. You may have a look for notebooks on the need with a deeper analysis of the data set itself.

In [ ]:
# Parameters

# Data
trainRatio = 0.75

# Model
numEst = 150
spliCrit = 'gini'
maxLeafNodes = 20
outBagScore = True

# Feature Permutation
numRepeats = 50


## Generate / Load Data

The data is based on the [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic).  

* <font color='brown'>(**#**)</font> The [Titanic Data Description](https://www.kaggle.com/c/titanic/data).
* <font color='brown'>(**#**)</font> The [Extended Titanic Data Description](https://www.kaggle.com/datasets/ibrahimelsayed182/titanic-dataset).

In [ ]:
# Load Data

dfX, dsY = fetch_openml('titanic', version = 1, return_X_y = True, as_frame = True, parser = 'auto')

print(f'The features data shape: {dfX.shape}')
print(f'The labels data shape: {dsY.shape}')

### Plot Data

In [ ]:
# Plot the Data

# The Data Frame of Features
dfX.head(10)


* <font color='red'>(**?**)</font> What kind of a feature is `name`? Should it be used?
* <font color='red'>(**?**)</font> What kind of a feature is `ticket`? Should it be used?

# Simple Feature Engineering by the Ticket

The ticket will be used in a way it adds some information.  
We'd count the number of passengers on the same ticket.  
Assuming those passengers have some kind of relation.

In [ ]:
# Count the Number of Passenger on the Same Ticket Number
dMap              = dfX['ticket'].value_counts().to_dict() #<! Equivalent to `dfX.groupby('ticket').size().sort_values(ascending = False).to_dict()`
dfX['passTicket'] = dfX['ticket'].map(dMap)

## Pre Processing and EDA

* <font color='brown'>(**#**)</font> [Julia Silge](https://juliasilge.com) is known for her [EDA Tutorials](https://www.youtube.com/@JuliaSilge/videos) and [Blog Posts](https://juliasilge.com/blog).
* <font color='brown'>(**#**)</font> David Robinson is known for his [EDA Tutorials](https://www.youtube.com/@safe4democracy/videos).
* <font color='brown'>(**#**)</font> The [Machine Learning Sliced VLog](https://www.youtube.com/playlist?list=PL6PX3YIZuHhyQmXKnyZmVDzdgAYbzwgDw) shows the whole process of working with real world data with an emphasize on Exploratory Data Analysis (EDA).

In [ ]:
# Dropping Features
# Dropping the `name` and `ticket` features (Avoid 1:1 identification)

dfX = dfX.drop(columns = ['name', 'ticket'])

* <font color='brown'>(**#**)</font> Pay attention that we dropped the features, but a deeper analysis could extract information from them (Type of ticket, families, etc...). 

In [ ]:
# The Features Data Frame
dfX.head(10)

In [ ]:
# The Labels Data Series
dsY.head(10)

In [ ]:
# Merging Data
dfData = pd.concat([dfX, dsY], axis = 1)

In [ ]:
# Data Frame Info
dfData.info()

In [ ]:
# Missing / Invalid Values
# Null / NA / NaN Matrix

#===========================Fill This===========================#
# 1. Calculate the logical map of invalid values using the method `isna()`.
mInvData = dfData.isna() #<! The logical matrix of invalid values
#===============================================================#

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
sns.heatmap(data = mInvData, square = False, ax = hA)
hA.set_title('Invalid Data Map')

plt.show()

Since the features `cabin`, `boat`, `body` and `home.dest` have mostly non valid values the will be dropped as well.

* <font color='brown'>(**#**)</font> Some implementation of Ensemble Trees can handle missing values. They might benefit in such case asl well.

In [ ]:
# Features Filtering
# Removing Features with Invalid Values.

dfData = dfData.drop(columns = ['cabin', 'boat', 'body', 'home.dest'])

In [ ]:
# Data Visualization
# Basic EDA on the Data

numCol = dfData.shape[1]
lCols  = dfData.columns
numAx  = int(np.ceil(np.sqrt(numCol)))

# hIsCatLikData = lambda dsX: (pd.api.types.is_categorical_dtype(dsX) or pd.api.types.is_bool_dtype(dsX) or pd.api.types.is_object_dtype(dsX) or pd.api.types.is_integer_dtype(dsX)) #<! Deprecated
hIsCatLikData = lambda dsX: (isinstance(dsX.dtype, pd.CategoricalDtype) or pd.api.types.is_bool_dtype(dsX) or pd.api.types.is_object_dtype(dsX) or pd.api.types.is_integer_dtype(dsX))

hF, hAs = plt.subplots(nrows = numAx, ncols = numAx, figsize = (16, 12))
hAs = hAs.flat

for ii in range(numCol):
    colName = dfData.columns[ii]
    if hIsCatLikData(dfData[colName]):
        sns.histplot(data = dfData, x = colName, hue = 'survived', stat = 'count', discrete = True, common_norm = True, multiple = 'dodge', ax = hAs[ii])
    else:
        sns.kdeplot(data = dfData, x = colName, hue = 'survived', fill = True, common_norm = True, ax = hAs[ii])

plt.show()

* <font color='red'>(**?**)</font> Have a look on the features and try to estimate their importance for the estimation.

### Filling Missing Data

For the rest of the missing values we'll use a simple method of interpolation:

 - Categorical Data: Using the mode value.
 - Numeric Data: Using the median / mean.

* <font color='brown'>(**#**)</font> We could employ much more delicate and sophisticated data.  
For instance, use the mean value of the same `pclass`. Namely profiling the data by other features to interpolate the missing feature.
* <font color='brown'>(**#**)</font> Data imputing can be done by using a model as well: Regression for _continuous_ data, Classification for _categorical_ data.
* <font color='brown'>(**#**)</font> The relevant classed in SciKit Learn: [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html), [`IterativeImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html), [`KNNImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html).

In [ ]:
# Missing Value by Dictionary
dNaNs = {'embarked': dfData['embarked'].mode()[0], 'age': dfData['age'].median(), 'fare': dfData['fare'].median()}

dfData = dfData.fillna(value = dNaNs, inplace = False) #<! We can use the `inplace` for efficiency

* <font color='brown'>(**#**)</font> The above is equivalent of using [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html).
* <font color='red'>(**?**)</font> Explain the **Data Leakage** above. How can it be avoided?

In [ ]:
# Null / NA / NaN Matrix

mInvData = dfData.isna() #<! The logical matrix of invalid values

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
sns.heatmap(data = mInvData, square = False, ax = hA)
hA.set_title('Invalid Data Map')

plt.show()

### Conversion of Categorical Data

In this notebook we'll use the [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) implementation of Random Forest.  
At the moment, it doesn't support Categorical Data, hence we'll use Dummy Variables (One Hot Encoding).

* <font color='brown'>(**#**)</font> Pay attention that one hot encoding is an inferior approach to having a real support for categorical data.
* <font color='brown'>(**#**)</font> For 2 values categorical data (Binary feature) there is no need fo any special support.
* <font color='brown'>(**#**)</font> Currently, the implementation of [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) doesn't support categorical values which are not ordinal (As it treats them as numeric). Hence we must use `OneHotEncoder`.  
See status at https://github.com/scikit-learn/scikit-learn/pull/12866.
* <font color='brown'>(**#**)</font> The _One Hot Encoding_ is not perfect. See [Are Categorical Variables Getting Lost in Your Random Forest](https://web.archive.org/web/20200307172925/https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/) (Also [Notebook - Are Categorical Variables Getting Lost in Your Random Forest](https://notebook.community/roaminsight/roamresearch/BlogPosts/Categorical_variables_in_tree_models/categorical_variables_post)). 

In [ ]:
# Features Encoding
# 1. The feature 'embarked' -> One Hot Encoding.
# 1. The feature 'sex' -> Mapping (Female -> 0, Male -> 1).
dfData = pd.get_dummies(dfData, columns = ['embarked'], drop_first = False)
dfData['sex'] = dfData['sex'].map({'female': 0, 'male': 1})

In [ ]:
# Convert Data Type
dfData = dfData.astype(dtype = {'pclass': np.uint8, 'sex': np.uint8, 'sibsp': np.uint8, 'parch': np.uint8, 'survived': np.uint8})

## Train the Random Forest Model

The Random Forest models basically creates weak classifiers by limiting their access to the data and features.  
This basically also limits their correlation which means we can use their mean in order to reduce the variance of the estimation.

### Split Train & Test

In [ ]:
# Split to Features and Labels

dfX = dfData.drop(columns = ['survived'])
dsY = dfData['survived']

print(f'The features data shape: {dfX.shape}')
print(f'The labels data shape: {dsY.shape}')

In [ ]:
# Split the Data

dfXTrain, dfXTest, dsYTrain, dsYTest = train_test_split(dfX, dsY, train_size = trainRatio, random_state = seedNum, shuffle = True, stratify = dsY)


### Train the Model

In [ ]:
# Construct the Model & Train

oRndForestsCls = RandomForestClassifier(n_estimators = numEst, criterion = spliCrit, max_leaf_nodes = maxLeafNodes, oob_score = outBagScore)
oRndForestsCls = oRndForestsCls.fit(dfXTrain, dsYTrain)

In [ ]:
# Scores of the Model

print(f'The train accuracy     : {oRndForestsCls.score(dfXTrain, dsYTrain):0.2%}')
print(f'The out of bag accuracy: {oRndForestsCls.oob_score_:0.2%}')
print(f'The test accuracy      : {oRndForestsCls.score(dfXTest, dsYTest):0.2%}')

* <font color='blue'>(**!**)</font> Try different values for the model's hyper parameter (Try defaults as well).

## Feature Importance Analysis

### Contribution to Impurity Decrease

This is the default method for feature importance of decision trees based methods.  
It basically sums the amount of impurity reduced by splits by each feature.

In [ ]:
# Extract Feature Importance
vFeatImp = oRndForestsCls.feature_importances_
vIdxSort = np.argsort(vFeatImp)


In [ ]:
# Display Feature Importance
hF, hA = plt.subplots(figsize = (16, 8))
hA.bar(x = dfXTrain.columns[vIdxSort], height = vFeatImp[vIdxSort])
hA.set_title('Features Importance of the Model')
hA.set_xlabel('Feature Name')
hA.set_title('Importance')

plt.show()

* <font color='red'>(**?**)</font> Do we need all 3: `embarked_C`, `embarked_Q` and `embarked_S`? Look at the options of `get_dummies()`.

### Permutation Effect

This is a more general method to measure the importance of a feature.  
We basically replace the values with "noise" to see how much performance has been deteriorated.

* <font color='brown'>(**#**)</font> The SciKit Learn's function which automates the process is [`permutation_importance()`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html).
* <font color='brown'>(**#**)</font> This is highly time consuming operation. Hence the speed of decision trees based methods creates a synergy.
* <font color='brown'>(**#**)</font> The importance is strongly linked to the estimator in use.

In [ ]:
# Test Permutations
oFeatImpPermTrain = permutation_importance(oRndForestsCls, dfXTrain, dsYTrain, n_repeats = numRepeats)
oFeatImpPermTest  = permutation_importance(oRndForestsCls, dfXTest, dsYTest, n_repeats = numRepeats)

In [ ]:
# Generate Data Frame

dT = {'Feature': [], 'Importance': [], 'Data': []}

for (dataName, mScore) in [('Train', oFeatImpPermTrain.importances), ('Test', oFeatImpPermTest.importances)]:
    for ii, featName in enumerate(dfX.columns):
        for jj in range(numRepeats):
            dT['Feature'].append(featName)
            dT['Importance'].append(mScore[ii, jj])
            dT['Data'].append(dataName)

dfFeatImpPerm = pd.DataFrame(dT)
dfFeatImpPerm = dfFeatImpPerm.sort_values(by = ['Data', 'Importance'], ascending = False)
dfFeatImpPerm

In [ ]:
# Generate a List of Feature Names Ordered by Median

dsMedianFeature = dfFeatImpPerm[dfFeatImpPerm['Data'] == 'Train'].groupby('Feature')['Importance'].median()
lFeatOrderMed   = dsMedianFeature.sort_values(ascending = False).index.to_list() #<! Index is the feature name

In [ ]:
# Plot Results
hF, hA = plt.subplots(figsize = (12, 8))
sns.boxplot(data = dfFeatImpPerm, x = 'Feature', y = 'Importance', hue = 'Data', order = lFeatOrderMed, ax = hA)

plt.show()

* <font color='green'>(**@**)</font> Try extracting better results by using the dropped features.